In [33]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
def read_article(file):
  file = open(file,"r")
  data = file.readlines()
  article = data[0].split('. ')
  sentences = []
  for sentence in article:
    sentences.append(sentence.replace("[^a-zA-Z]"," ").split(" "))
  sentences.pop()
  return sentences

In [35]:
def similarity(s1,s2,stop_words = None):
  if stop_words is None :
    stop_words = None 
  s1 = [w.lower for w in s1]
  s2 = [w.lower for w in s2]
  all = list(set(s1+s2))
  v1 = [0]*len(all)
  v2 = [0]*len(all)
  for w in s1:
    if w in stop_words:
      continue
    v1[all.index(w)] += 1
  for w in s2:
    if w in stop_words:
      continue
    v2[all.index(w)] += 1
  return 1 - cosine_distance(v1, v2)


In [36]:
def matrix(sentence,stop_words):
  similar_matrix = np.zeros((len(sentence),len(sentence)))
  for s1 in range(len(sentence)):
    for s2 in range(len(sentence)):
      if s1 == s2:
        continue
      similar_matrix[s1][s2] = similarity(sentence[s1],sentence[s2],stop_words)
  return similar_matrix


In [37]:
def summary(file, top_n = 5):
  stop_word = stopwords.words('english')
  summarize_text = []
  sentence = read_article(file)
  sen_simil_matrix = matrix(sentence, stop_word)
  sen_simil_graph = nx.from_numpy_array(sen_simil_matrix)
  scores = nx.pagerank(sen_simil_graph)
  ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentence)),reverse = True)
  for i in range(top_n):
    summarize_text.append(" ".join(ranked_sentence[i][1]))
  print("Summary:- \n",". ".join(summarize_text))

In [38]:
no_of_lines_you_want = 5
summary('/content/sample_data/test.txt',no_of_lines_you_want)

Summary:- 
 Tillie Olsen's "I Stand Here Ironing" (1961) adopted a consciously feminist perspective. Traditionalists including John Updike and Joyce Carol Oates maintained a significant influence on the form. Cultural and social identity played a considerable role in much of the short fiction of the 1960s. The period following World War II saw a great flowering of literary short fiction in the United States. Wallace Stegner's short stories are primarily set in the American West
